# Project 4

In this project, you will summarize and present your analysis from Projects 1-3.

### Intro: Write a problem Statement/ Specific Aim for this project

Answer: Use the gpa, gre, and school prestige to predict the likehood of admission to UCLA's graduate program.

### Dataset:  Write up a description of your data and any cleaning that was completed

* 'admit' is a binary variable. It indicates whether or not a candidate was admitted admit =1) our not (admit= 0)
* 'gre' is GRE score
* 'gpa' stands for Grade Point Average
* 'rank' is the rank of an applicant's undergraduate alma mater, with 1 being the highest and 4 as the lowest

Answer: Remove null values from all rows.

### Demo: Provide a table that explains the data by admission status

Mean (STD) or counts by admission status for each variable 

| Not Admitted | Admitted
---| ---|---
GPA | mean(std)  | mean(std)
GRE |mean(std) | mean(std)
Prestige 1 | frequency (%) | frequency (%)
Prestige 2 | frequency (%) | frequency (%)
Prestige 3 |frequency (%) | frequency (%)
Prestige 4 |frequency (%) | frequency (%)

In [3]:
#imports
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
import numpy as np
%matplotlib inline

In [29]:
#Read in data from source 
df_raw = pd.read_csv("../admissions.csv")
df_clean = df_raw.dropna()


In [85]:
gre_admit = df_clean[(df_clean.admit == 1)]['gre']
gre_admit_mean = gre_admit.mean()
gre_admit_std = gre_admit.std()

gre_reject = df_clean[(df_clean.admit == 0)]['gre']
gre_reject_mean = gre_reject.mean()
gre_reject_std = gre_reject.std()

gpa_admit = df_clean[(df_clean.admit == 1)]['gpa']
gpa_admit_mean = gpa_admit.mean()
gpa_admit_std = gpa_admit.std()

print(gpa_admit_std)

gpa_reject = df_clean[(df_clean.admit == 0)]['gpa']
gpa_reject_mean = gpa_reject.mean()
gpa_reject_std = gpa_reject.std()

0.37165490051843125


In [86]:
p1_admit_freq=df_clean[(df_clean.prestige == 1.0) & (df_clean.admit == 1)].size/df_clean[df_clean.prestige == 1.0].size
p2_admit_freq=df_clean[(df_clean.prestige == 2.0) & (df_clean.admit == 1)].size/df_clean[df_clean.prestige == 2.0].size
p3_admit_freq=df_clean[(df_clean.prestige == 3.0) & (df_clean.admit == 1)].size/df_clean[df_clean.prestige == 3.0].size
p4_admit_freq=df_clean[(df_clean.prestige == 4.0) & (df_clean.admit == 1)].size/df_clean[df_clean.prestige == 4.0].size

p1_reject_freq=df_clean[(df_clean.prestige == 1.0) & (df_clean.admit == 0)].size/df_clean[df_clean.prestige == 1.0].size
p2_reject_freq=df_clean[(df_clean.prestige == 2.0) & (df_clean.admit == 0)].size/df_clean[df_clean.prestige == 2.0].size
p3_reject_freq=df_clean[(df_clean.prestige == 3.0) & (df_clean.admit == 0)].size/df_clean[df_clean.prestige == 3.0].size
p4_reject_freq=df_clean[(df_clean.prestige == 4.0) & (df_clean.admit == 0)].size/df_clean[df_clean.prestige == 4.0].size

In [87]:
print('| Not Admitted | Admitted')
print('GPA | {:.2f} ({:.2f}) |{:.2f} ({:.2f}) '.format(gpa_reject_mean, gpa_reject_std, gpa_admit_mean, gpa_admit_std))
print('gre | {:.2f} ({:.2f}) |{:.2f} ({:.2f}) '.format(gre_reject_mean, gre_reject_std, gre_admit_mean, gre_admit_std))
print('prestige 1 | {0:.2f} |{1:.2f} ) '.format(p1_reject_freq,p1_admit_freq))
print('prestige 2 | {0:.2f} |{1:.2f} ) '.format(p2_reject_freq,p2_admit_freq))
print('prestige 3 | {0:.2f} |{1:.2f} ) '.format(p3_reject_freq,p3_admit_freq))
print('prestige 4 | {0:.2f} |{1:.2f} ) '.format(p4_reject_freq,p4_admit_freq))

| Not Admitted | Admitted
GPA | 3.35 (0.38) |3.49 (0.37) 
gre | 573.58 (116.05) |618.57 (109.26) 
prestige 1 | 0.46 |0.54 ) 
prestige 2 | 0.64 |0.36 ) 
prestige 3 | 0.77 |0.23 ) 
prestige 4 | 0.82 |0.18 ) 


### Methods: Write up the methods used in your analysis

Answer: Summary analysis, correlation matrix, cross tab, logistic regression

### Results: Write up your results

Answer: According to our dataset, the best measure of whether students will be admitted to graduate school is the prestige of their undergraduate institution.

The table above shows a good approximation: 46% of prestige 1 students are admitted, whereas over 80% of prestige 4 studentd are rejected.  By comparison, the GPA and gre average of rejected vs accepted students is quite narrow.

### Visuals: Provide a table or visualization of these results

In [168]:
# Prepare the data for a regression:
dummies = pd.get_dummies(df_clean['prestige'], prefix='prestige')
dummies.head()

cols_to_keep = ["admit","gre","gpa"]
data = df_clean[cols_to_keep].join(dummies.ix[:, 'prestige_2':])
# manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,0,380.0,3.61,0.0,1.0,0.0,1.0
1,1,660.0,3.67,0.0,1.0,0.0,1.0
2,1,800.0,4.00,0.0,0.0,0.0,1.0
3,1,640.0,3.19,0.0,0.0,1.0,1.0
4,0,520.0,2.93,0.0,0.0,1.0,1.0


In [169]:
logit = sm.Logit(data['admit'],data[data.columns[1:]])
results = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


In [171]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 18 Jul 2016   Pseudo R-squ.:                 0.08166
Time:                        17:50:27   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
intercept       -3.8769      1.142     -3.393      0.001        -6.116    -1.638
================================================================================
"""

### Discussion: Write up your discussion and future steps

Answer: GRE scores do not have a major impact on admission status. It seems that the school's prestige is the most significant predictor, followed by the GPA.  